In [ ]:
## NOTe that HELA never converged for GEMINI
# Uncomment the command below if you run this script separately. 

#%run AnalysisFunctions.ipynb


In [ ]:
print("Running Parrish Analysis")


print(f"Apply Filter?: {apply_filter}")

if apply_filter.lower() == "true":
    filtered = "/Filtered"
    print("Compiling results with filtered pairs")
else:
    filtered = ""
    print("Compiling results with unfiltered pairs")

In [ ]:
# In this file, I will analyses calculated scores for the Parrish screen

# Start loading calculated scores 
# For zdLFC, filtered is same as unfiltered because it doesnt filter out anything
zdLFC_Hela = pd.read_csv("../zdLFC Scripts/zdLFC Output/Parrish_Hela.csv")
zdLFC_Hela = zdLFC_Hela.rename(columns={zdLFC_Hela.columns[0]: 'GENE'})
zdLFC_Hela = zdLFC_Hela.set_index('GENE')


zdLFC_PC9 = pd.read_csv("../zdLFC Scripts/zdLFC Output/Parrish_PC9.csv")
zdLFC_PC9 = zdLFC_PC9.rename(columns={zdLFC_PC9.columns[0]: 'GENE'})
zdLFC_PC9 = zdLFC_PC9.set_index('GENE')

zdLFC = pd.concat([zdLFC_Hela, zdLFC_PC9], axis = 1)
zdLFC = -1 * zdLFC
zdLFC


In [ ]:
orthrus = pd.read_csv("../Orthrus Scripts/OrthrusOutput" + filtered + "/parrish_orthrus.tsv", sep='\t')
orthrus['GENE'] = orthrus['gene1'] + "_" + orthrus['gene2']
orthrus = orthrus.filter(like='differential', axis=1).join(orthrus.filter(items=['GENE']))
orthrus = orthrus.rename(columns={'differential_combn_vs_single_PC9': 'PC9', 
                                  'differential_combn_vs_single_HeLa': 'HeLa'})
orthrus = orthrus.set_index('GENE')
orthrus = reindex_alphbetically(orthrus)
orthrus = orthrus * -1
orthrus


In [ ]:
Hela_gemini_Sens_L = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Parrish_HeLa_Sensitive.csv")
Hela_gemini_Sens_L.rename({ "Unnamed: 0": "gene_pair"}, inplace = True, axis = 1)
Hela_gemini_Sens_L['gene_pair'] = Hela_gemini_Sens_L['gene_pair'].replace(':', '_', regex=True)
Hela_gemini_Sens_L.set_index('gene_pair', inplace = True)
Hela_gemini_Sens_L  = reindex_alphbetically(Hela_gemini_Sens_L )

display(Hela_gemini_Sens_L) 



Hela_gemini_Strong = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Parrish_HeLa_Strong.csv")
Hela_gemini_Strong.rename({ "Unnamed: 0": "gene_pair"}, inplace = True, axis = 1)
Hela_gemini_Strong['gene_pair'] = Hela_gemini_Strong['gene_pair'].replace(':', '_', regex=True)
Hela_gemini_Strong.set_index('gene_pair', inplace = True)

Hela_gemini_Strong  = reindex_alphbetically(Hela_gemini_Strong )#


display(Hela_gemini_Strong) 

PC9_gemini_Sens_L = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Parrish_PC9_Sensitive.csv")
PC9_gemini_Sens_L.rename({ "Unnamed: 0": "gene_pair"}, inplace = True, axis = 1)
PC9_gemini_Sens_L['gene_pair'] = PC9_gemini_Sens_L['gene_pair'].replace(':', '_', regex=True)
PC9_gemini_Sens_L.set_index('gene_pair', inplace = True)
PC9_gemini_Sens_L  = reindex_alphbetically(PC9_gemini_Sens_L )

PC9_gemini_Sens_L 



PC9_gemini_Strong = pd.read_csv("../Gemini Scripts/GeminiOutput" + filtered + "/Gemini_Parrish_PC9_Strong.csv")
PC9_gemini_Strong.rename({ "Unnamed: 0": "gene_pair"}, inplace = True, axis = 1)
PC9_gemini_Strong['gene_pair'] = PC9_gemini_Strong['gene_pair'].replace(':', '_', regex=True)
PC9_gemini_Strong.set_index('gene_pair', inplace = True)

PC9_gemini_Strong  = reindex_alphbetically(PC9_gemini_Strong )#
PC9_gemini_Strong 

In [ ]:
# Parrish doesnt filter out any pairs so filtered is same as unfiltered
parrish = pd.read_csv("../Parrish Score Scripts/ParrishOutput/Parrish_Parrish.csv")
parrish.set_index('paralog_pair', inplace = True)
parrish  = reindex_alphbetically(parrish )#

parrish = parrish * -1
parrish


In [ ]:
# HELA
Parrish_HeLa = pd.concat([zdLFC['HeLa'],
                          orthrus['HeLa'],
                          Hela_gemini_Strong['HeLa_LTP'],
                          Hela_gemini_Sens_L['HeLa_LTP'],
                          parrish['HeLa']], axis = 1, keys=scores)

results_Parrish_HeLa = Cell_Line_Analysis(Parrish_HeLa, 'HeLa', 'Parrish', 
                   scores,ground_truth_depmap_hit)



In [ ]:
# PC9
Parrish_PC9 = pd.concat([zdLFC['PC9'],
                          orthrus['PC9'],                          
                          PC9_gemini_Strong['PC9_LTP'],
                            PC9_gemini_Sens_L['PC9_LTP'],
                          parrish['PC9']
                                  
                      ], axis = 1, keys=scores)
results_Parrish_PC9 = Cell_Line_Analysis(Parrish_PC9, 'PC9', 'Parrish', 
                   scores, ground_truth_depmap_hit)



In [ ]:
full_list = [results_Parrish_PC9, results_Parrish_HeLa]

roc, pr = ConvertResultsToDF(full_list )
roc
## Write results to a file
roc.to_csv("Output" + filtered + "/Parrish_ROC_DepMapHits.csv", index=False)
pr.to_csv("Output"  + filtered +  "/Parrish_PR_DepMapHits.csv", index=False)


In [ ]:
parrish_all = pd.concat([Parrish_HeLa, Parrish_PC9])
results_parrish_all = Cell_Line_Analysis(parrish_all, 'All', 'Parrish', 
                   scores, ground_truth_depmap_hit)

roc_all, pr_all = ConvertResultsToDF([results_parrish_all] )
## Write results to a file
roc_all.to_csv("Output" + filtered + "/Parrish_All_ROC_DepMapHits.csv", index=False)
pr_all.to_csv("Output" + filtered+ "/Parrish_All_PR_DepMapHits.csv", index=False)


In [ ]:
results_Parrish_PC9_Koferle = Cell_Line_Analysis(Parrish_PC9, 'PC9', 'Parrish', 
                   scores, Koferle)
results_Parrish_HeLa_Koferle = Cell_Line_Analysis(Parrish_HeLa, 'HeLa', 'Parrish', 
                   scores,Koferle)

full_list = [results_Parrish_PC9_Koferle, results_Parrish_HeLa_Koferle]

roc_Koferle, pr_Koferle = ConvertResultsToDF(full_list)

## Write results to a file
roc_Koferle.to_csv("Output" + filtered + "/Parrish_ROC_Koferle.csv", index=False)
pr_Koferle.to_csv("Output" + filtered + "/Parrish_PR_Koferle.csv", index=False)


In [ ]:
# combine all cell lines and then repeat analysis
results_parrish_all_Koferle = Cell_Line_Analysis(parrish_all, 'All', 'Parrish', scores, Koferle)

roc_all_Koferle, pr_all_Koferle= ConvertResultsToDF([results_parrish_all_Koferle] )
## Write results to a file
roc_all_Koferle.to_csv("Output" + filtered + "/Parrish_All_ROC_Koferle.csv", index=False)
pr_all_Koferle.to_csv("Output" + filtered + "/Parrish_All_PR_Koferle.csv", index=False)


In [ ]:
parrish_all.corr(method = "spearman").to_csv("Output" + filtered + "\Comparison\Parrish_Correlations.csv")
Parrish_PC9_ = jaccard_similarity(Parrish_PC9, 0.95)
Parrish_PC9_.to_csv("Output" + filtered + "\Comparison\Parrish_PC9_Jaccard.csv")
Parrish_HeLa_ = jaccard_similarity(Parrish_HeLa, 0.95)
Parrish_HeLa_.to_csv("Output" + filtered + "\Comparison\Parrish_HeLa_Jaccard.csv")

Parrish_Jaccard = (Parrish_HeLa_ + Parrish_HeLa_) / 2
Parrish_Jaccard.to_csv("Output" + filtered + "\Comparison\Parrish_Jaccard.csv")